# Previsão megasena com Keras

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas              as pd
import numpy               as np

## Carrega dados do arquivo *.csv

In [2]:
# Le arquivo de dados# the histogram of the data
df = pd.read_csv("resultados_megasena.csv")

# Gera lista com todas dezenas
dezenas = pd.DataFrame(df['1ª Dezena'].tolist() + df['2ª Dezena'].tolist() + df['3ª Dezena'].tolist() + df['4ª Dezena'].tolist() + df['5ª Dezena'].tolist() + df['6ª Dezena'].tolist(), columns=['numeros'])

sequence = pd.DataFrame(df[['1ª Dezena','2ª Dezena','3ª Dezena','4ª Dezena','5ª Dezena','6ª Dezena',]]).to_numpy()

df

,Unnamed: 0,Concurso,Data Sorteio,1ª Dezena,2ª Dezena,3ª Dezena,4ª Dezena,5ª Dezena,6ª Dezena,Arrecadacao_Total,...,UF,Rateio_Sena,Ganhadores_Quina,Rateio_Quina,Ganhadores_Quadra,Rateio_Quadra,Acumulado,Valor_Acumulado,Estimativa_Prêmio,Acumulado_Mega_da_Virada
0,0,1,11/03/1996,41,5,4,52,30,33,000,...,&nbsp,000,17,39.15892,2016,33021.0,SIM,1.714.65023,000,000
1,1,2,18/03/1996,9,39,37,49,43,41,000,...,PR,2.307.16223,65,14.42402,4488,20891.0,NÃO,000,000,000
2,2,3,25/03/1996,36,30,10,11,29,47,000,...,RN,391.19251,62,10.51593,4261,15301.0,NÃO,000,000,000
3,4,4,01/04/1996,6,59,42,27,1,5,000,...,&nbsp,000,39,15.32224,3311,18048.0,SIM,717.08075,000,000
4,5,5,08/04/1996,1,19,46,6,16,2,000,...,&nbsp,000,98,5.31810,5399,9653.0,SIM,1.342.48885,000,000
5,6,6,15/04/1996,19,40,7,13,22,47,000,...,&nbsp,000,109,7.21466,7147,11003.0,SIM,2.286.16633,000,000
6,7,7,22/04/1996,56,38,21,20,3,5,000,...,&nbsp,000,100,8.74605,5736,15248.0,SIM,3.335.69228,000,000
7,8,8,29/04/1996,53,17,38,4,47,37,000,...,&nbsp,000,60,16.08411,5262,18340.0,SIM,4.493.74819,000,000
8,9,9,06/05/1996,55,43,56,54,8,60,000,...,&nbsp,000,17,60.04379,2175,46931.0,SIM,5.718.64149,000,000
9,10,10,13/05/1996,25,4,18,57,21,38,000,...,&nbsp,000,251,6.63845,12590,13235.0,SIM,13.334.76981,000,000


# RNN Simples

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [4]:
n_steps = 40
n_inputs = 6
n_neurons = 100
n_outputs = 6
learning_rate = 0.001
batch_size = 50
n_epochs = 40
n_iterations = 50000
batch_size = 1000
i_min, i_max = 0, len(sequence)-n_steps-1

def time_series(i):
    return sequence[i,:]

# falta incorporar batch_size aqui
def next_batch(batch_size, n_steps):
    i0 = np.random.randint(i_max,size=(batch_size,1))
    Is = i0 + np.arange(0., n_steps + 1)
    ys = time_series(Is.astype(int))
    return ys[:, :-1].reshape(-1, n_steps, n_inputs), ys[:, 1:].reshape(-1, n_steps, n_inputs)

## Dados de entrada

In [5]:
X_series,y_series = next_batch(batch_size, n_steps)
X_train, y_train = X_series[:700, :n_steps], y_series[:700]
#X_valid, y_valid = X_series[700:900, :n_steps], y_series[700:900]
X_valid, y_valid = X_series[-1:, :n_steps], y_series[-1:]

X_pred = sequence[-n_steps:,:].reshape(-1, n_steps, n_inputs)

## Modelo

In [6]:
model = tf.keras.Sequential([
    keras.layers.SimpleRNN(n_neurons, return_sequences=True, input_shape=[None, n_inputs], activation="relu"),
    #keras.layers.GRU(20, return_sequences=True),
    #keras.layers.LSTM(20, return_sequences=True),
    keras.layers.Dense(6)
])
optimizer = keras.optimizers.Adam(lr=learning_rate)
model.compile(loss="mse",optimizer=optimizer)
model.summary()

W0905 15:10:26.688933 140295210153792 deprecation.py:506] From /home/jairo/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, None, 100)         10700     
_________________________________________________________________
dense (Dense)                (None, None, 6)           606       
Total params: 11,306
Trainable params: 11,306
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(X_train, y_train, epochs=n_epochs, 
                   validation_data=(X_valid, y_valid))

Train on 700 samples, validate on 1 samples
Epoch 1/40
700/700 [==============================] - 2s 3ms/sample - loss: 562.5318 - val_loss: 350.7285
Epoch 2/40
700/700 [==============================] - 1s 1ms/sample - loss: 345.6840 - val_loss: 332.5138
Epoch 3/40
700/700 [==============================] - 1s 2ms/sample - loss: 329.8933 - val_loss: 320.1535
Epoch 4/40
700/700 [==============================] - 1s 2ms/sample - loss: 320.2618 - val_loss: 307.7693
Epoch 5/40
700/700 [==============================] - 1s 2ms/sample - loss: 312.1151 - val_loss: 295.7696
Epoch 6/40
700/700 [==============================] - 1s 2ms/sample - loss: 306.2536 - val_loss: 288.0667
Epoch 7/40
700/700 [==============================] - 1s 2ms/sample - loss: 302.0634 - val_loss: 282.1249
Epoch 8/40
700/700 [==============================] - 1s 2ms/sample - loss: 299.2168 - val_loss: 282.1022
Epoch 9/40
700/700 [==============================] - 1s 2ms/sample - loss: 297.6357 - val_loss: 280.8581
Ep

## Previsão

In [8]:
y_pred = model.predict(X_pred)
y_pred[0,-1:,:]

array([[27.369394, 33.554085, 25.626486, 28.784794, 29.093685, 38.55515 ]],
      dtype=float32)